In [1]:
import pandas as pd

In [4]:
ask_asc = pd.read_csv('../../data/ai_responses/ask_reddit_science_final.csv')
remaining = pd.read_csv('../../data/ai_responses/remaining_subs_final.csv')

In [8]:
df = pd.concat([ask_asc, remaining]).reset_index(drop = True)

In [9]:
df.shape

(5413, 4)

In [10]:
df.head()

,subreddit,title,comment,ai_response
0,AskReddit,Would you support a mandatory retirement age o...,70 and as for president no one can run over 65...,"No, I don't think a mandatory retirement age o..."
1,AskReddit,Now that Reddit are killing 3rd party apps on ...,"Without this app, I will actually stop using R...",I'm not sure what the best alternatives to Red...
2,AskReddit,Bernie Sanders Says US Should Confiscate 100% ...,So maybe someone can try to explain this a bit...,I think Bernie Sanders' suggestion is extreme ...
3,AskReddit,"Richard Feynman said, “Never confuse education...",My wife's stepfather was a chemist who current...,I think a great example of this is people who ...
4,AskReddit,How much would you pay for a list of everyone ...,"$20. Realistically it would be interesting, bu...",I wouldn't pay anything for a list like that. ...


In [11]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5413 entries, 0 to 5412
Data columns (total 4 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   subreddit    5413 non-null   object
 1   title        5413 non-null   object
 2   comment      5413 non-null   object
 3   ai_response  5413 non-null   object
dtypes: object(4)
memory usage: 169.3+ KB


In [8]:
#df.to_csv('../../data/data_merged.csv', index = False)

In [15]:
df['subreddit'].value_counts()

DoesAnybodyElse      976
NoStupidQuestions    960
AskReddit            958
explainlikeimfive    919
AskScience           824
AskHistorians        776
Name: subreddit, dtype: int64

In [16]:
df['subreddit'].value_counts(normalize = True)

DoesAnybodyElse      0.180307
NoStupidQuestions    0.177351
AskReddit            0.176981
explainlikeimfive    0.169776
AskScience           0.152226
AskHistorians        0.143359
Name: subreddit, dtype: float64

In [29]:
df[df['ai_response'].str.contains('language model')] # Looks like no answers of form "As an AI language model"

,subreddit,title,comment,ai_response


In [30]:
df[df['ai_response'].str.contains('ChatGPT')] # Only mentions of chatGPT seem safe

,subreddit,title,comment,ai_response
3325,explainlikeimfive,ELI5: why do models like ChatGPT forget things...,"Very simply, they don't know anything about th...",That's because ChatGPT is a form of artificial...
3419,explainlikeimfive,Eli5: how does software that can detect text g...,Anomaly detection is the machine learning meth...,Software that uses ChatGPT works by using a ty...


### Creating a dataset with only subreddit, responses, and a variable for real or fake

In [42]:
ai_responses = pd.concat([df['subreddit'], df['ai_response']], axis = 1) 

In [35]:
ai_responses.shape

(5413, 2)

In [36]:
real_responses = pd.concat([df['subreddit'], df['comment']], axis = 1)

In [37]:
real_responses.shape

(5413, 2)

In [43]:
ai_responses['fake'] = [1]*ai_responses.shape[0]
real_responses['fake'] = [0]*real_responses.shape[0]

In [44]:
ai_responses.rename(columns = {'subreddit' : 'subreddit', 'ai_response': 'response'}, inplace = True)
real_responses.rename(columns = {'subreddit' : 'subreddit', 'comment' : 'response'}, inplace = True)

In [48]:
modelling_ready = pd.concat([ai_responses, real_responses])

In [49]:
modelling_ready.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 10826 entries, 0 to 5412
Data columns (total 3 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   subreddit  10826 non-null  object
 1   response   10826 non-null  object
 2   fake       10826 non-null  int64 
dtypes: int64(1), object(2)
memory usage: 338.3+ KB


In [50]:
modelling_ready.head()

,subreddit,response,fake
0,AskReddit,"No, I don't think a mandatory retirement age o...",1
1,AskReddit,I'm not sure what the best alternatives to Red...,1
2,AskReddit,I think Bernie Sanders' suggestion is extreme ...,1
3,AskReddit,I think a great example of this is people who ...,1
4,AskReddit,I wouldn't pay anything for a list like that. ...,1


In [51]:
modelling_ready['fake'].value_counts(normalize = True)

1    0.5
0    0.5
Name: fake, dtype: float64

In [52]:
#modelling_ready.to_csv('../../data_final.csv', index = False)

In [3]:
modelling_ready['response'].is_unique

False

In [10]:
modelling_ready[modelling_ready['response'].duplicated()] # The one duplicated response is the duplicated one from before

,subreddit,response,fake
6088,AskReddit,Ticketmaster,0
